In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import metrics
import matplotlib.pyplot as plt

# **Data Exploration**

**1. Data Training**

In [ ]:
df = pd.read_excel('/content/gdrive/My Drive/BDC SD/Salinan Data Latih BDC.xlsx')

In [ ]:
df.head(10)

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
5,736,1,2020-06-17 00:00:00,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma...",736.png
6,1161,1,2020-05-05 00:00:00,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...,1161.png
7,1597,1,2020-03-24 00:00:00,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...,1597.png
8,2098,1,2020-01-09 00:00:00,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...,2098.jpg
9,2226,1,2019-12-24 00:00:00,Video Polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...,2226.png


In [ ]:
df.shape

(4231, 6)

In [ ]:
df = df.drop(['tanggal', 'nama file gambar'], axis=1)

In [ ]:
df = df.set_index('ID')

In [ ]:
df.head(10)

,label,judul,narasi
ID,,,
71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .
736,1,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma..."
1161,1,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...
1597,1,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...
2098,1,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4231 entries, 71 to 999860
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   4231 non-null   int64 
 1   judul   4231 non-null   object
 2   narasi  4231 non-null   object
dtypes: int64(1), object(2)
memory usage: 132.2+ KB


In [ ]:
df['label'].value_counts()

1    3465
0     766
Name: label, dtype: int64

Dari label, terlihat data unbalanced karena label hoax(1) jauh lebih banyak daripada label 0.

Menggabungkan judul dan narasi menjadi kolom baru


In [ ]:
df['total']=df['judul'] + ' ' + df['narasi']
df.head()

,label,judul,narasi,total
ID,,,,
71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,Pemakaian Masker Menyebabkan Penyakit Legionna...
461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,Instruksi Gubernur Jateng tentang penilangan ...
495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...","ini bukan politik, tapi kenyataan Pak Jokowi b..."
681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,Foto Kadrun kalo lihat foto ini panas dingin K...


**2. Data Testing**

In [ ]:
df_t = pd.read_excel('/content/gdrive/My Drive/BDC SD/Data Uji BDC.xlsx')
df_t.head()

,ID,tanggal,judul,narasi,nama file gambar,Unnamed: 5
0,238057,2020-07-13 00:00:00,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg,1
1,238158,2020-07-06 00:00:00,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg,1
2,238865,2020-04-22 00:00:00,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg,1
3,248298,2019-10-22 00:00:00,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg,1
4,255176,2020-05-01 00:00:00,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg,1


In [ ]:
df_t['total']=df_t['judul'] + ' ' + df_t['narasi']
df_t = df_t.drop(['tanggal', 'nama file gambar'], axis=1)
df_t.head()

,ID,judul,narasi,Unnamed: 5,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,Narasi Tito Karnavian Berideologi Komunis Kare...
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,Anies: Seberat beratnya Pekerjaan Akan terasa ...
2,238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,Hindu di india Melemparkan Patung Buatan Merek...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,Permohonan Kelonggaran Angsuran ke OJK Untuk s...


In [ ]:
df_t = df_t.set_index('ID')

# **Preprocessing Data**

In [ ]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install Sastrawi
#stopwords removal
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop = StopWordRemoverFactory().get_stop_words()

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()

     |████████████████████████████████| 215kB 2.8MB/s 


In [ ]:
stemmer = factory.create_stemmer()

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
stop_words = list_stopwords

#preprocessing data training
for index, row in df.iterrows():
    filter_sentence = ''
    sentence = row['total']
    # Cleaning dengan regular expression
    sentence = re.sub(r'[^\w\s]', '', sentence)
    sentence = stemmer.stem(sentence)
    # Tokenization
    words = nltk.word_tokenize(sentence)
    # Menghilangkan stopwords
    words = [w for w in words if not w in stop_words]
    # Lower case
    for words in words:
        filter_sentence = filter_sentence  + ' ' + words.lower()
    
    df.loc[index, 'total'] = filter_sentence

#preprocessing data testing    
for index, row in df_t.iterrows():
    filter_sentence = ''
    sentence = row['total']
    # Cleaning dengan regular expression
    sentence = re.sub(r'[^\w\s]', '', sentence)
    sentence = stemmer.stem(sentence)
    # Tokenization
    words = nltk.word_tokenize(sentence)
    # Menghilangkan stopwords
    words = [w for w in words if not w in stop_words]
    # Lower case
    for words in words:
        filter_sentence = filter_sentence  + ' ' + words.lower()
    
    df_t.loc[index, 'total'] = filter_sentence

In [ ]:
df.head(10)

In [ ]:
#inisiasi x dan y
x_train = df['total']
y_train = df['label']

In [ ]:
df.to_excel('datat.xlsx')
!cp datat.xlsx "/content/gdrive/My Drive/BDC SD"

In [ ]:
df_t.head()

In [ ]:
x_test = df_t['total']
y_test = df_t['label']

In [ ]:
df_t.to_excel('data2xu.xlsx')
!cp data2xu.xlsx "/content/gdrive/My Drive/BDC SD"

# **Vectorization**

1. TF-IDF Vectorization

In [ ]:
# Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(max_df=1.0)

#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

2. Count Vectorizer

In [ ]:
# Initialize a CountVectorizer
count_vectorizer = CountVectorizer()

#Fit and transform train set, transform test set
count_train = count_vectorizer.fit_transform(x_train)
count_test = count_vectorizer.transform(x_test)

# **Modelling**

1. PASSIVE-AGGRESSIVE CLASSIFIER

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
# PASSIVE-AGGRESSIVE CLASSIFIER TF-IDF   

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
#Predict on the test set
y_pred = pac.predict(tfidf_test)

#F1 Score
scoref1 = f1_score(y_test, y_pred)
print(f'F1 Score: {round(scoref1*100,2)}%')


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred, labels=[0,1])
print(confusion_matrix)

In [ ]:
# PASSIVE-AGGRESSIVE CLASSIFIER Count

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(count_train,y_train)

#Predict on the test set
y_pred=pac.predict(count_test)

scoref1c=f1_score(y_test, y_pred)
print(f'F1 score: {round(scoref1c*100,2)}%')

2. MULTINOMIAL NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
#Multinomial NB TF-IDF

# Create the list of alphas: alphas
alphas = np.arange(0, 1, 0.1)

# Define train_and_predict()
def train_and_predict(alpha, tfidf_train, y_train, tfidf_test, y_test):
    # Instantiate the classifier: nb_classifier
    nb_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_classifier.fit(tfidf_train, y_train)
    # Predict the labels: pred
    pred = nb_classifier.predict(tfidf_test)
    # Compute accuracy: score
    score = metrics.f1_score(y_test, pred)
    return score

# Iterate over the alphas and print the corresponding score
for alpha in alphas:
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha, tfidf_train, y_train, tfidf_test, y_test))
    print()

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(tfidf_train, y_train)
Accuracy = logreg.score(tfidf_test, y_test)

print(Accuracy*100)

In [ ]:
hasil = pd.DataFrame()